In [15]:
%load_ext autoreload
%autoreload 2

In [1]:
import os
import sys
sys.path.append("..")  # add top folder to path

import moses
import gentrl

import tinymolecule as tiny

In [7]:
enc = tiny.LinearEncoder(latent_size=10)  # mu and sigma
dec = tiny.LinearDecoder(latent_size=5)  # a single sample
vae = tiny.LinearVAE(enc, dec)

In [8]:
train = moses.get_dataset('train')

In [9]:
train_encoded = gentrl.tokenizer.encode(train[1:100])[0].float()

In [10]:
train_encoded[0:2]

tensor([[ 1., 23., 23., 22., 23., 15., 22., 23., 15., 23., 22., 13.,  9., 15.,
         23., 22.,  9., 10., 24., 10., 10., 10., 22.,  5., 15., 10., 10., 24.,
         15., 18., 24., 10., 10., 18., 10., 24.,  2.,  2.,  2.,  2.,  2.,  2.,
          2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.],
        [ 1., 23., 10., 24., 10., 22.,  5., 15., 10., 10., 10., 10., 24.,  6.,
         10., 24., 18., 10., 10., 10., 10., 24., 23., 22., 13.,  9., 15.,  9.,
         23., 23., 22.,  9., 15., 23.,  9.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,
          2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.]])

In [11]:
vae.forward(train_encoded[0:1])

RuntimeError: size mismatch, m1: [1 x 25], m2: [5 x 100] at /Users/distiller/project/conda/conda-bld/pytorch_1595629444482/work/aten/src/TH/generic/THTensorMath.cpp:41

In [29]:
train_encoded[0]

tensor([ 1, 23, 23, 22, 23, 15, 22, 23, 15, 23, 22, 13,  9, 15, 23, 22,  9, 10,
        24, 10, 10, 10, 22,  5, 15, 10, 10, 24, 15, 18, 24, 10, 10, 18, 10, 24,
         2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2])

In [6]:
vae.decoder

LinearDecoder(
  (layers): ModuleList(
    (0): Linear(in_features=10, out_features=100, bias=True)
    (1): Linear(in_features=100, out_features=100, bias=True)
    (2): Linear(in_features=100, out_features=10, bias=True)
  )
  (dropout): ModuleList(
    (0): Dropout(p=0.85, inplace=False)
    (1): Dropout(p=0.85, inplace=False)
  )
)